In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
patients_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/patients.csv")


In [0]:
patients_df.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- service: string (nullable = true)
 |-- satisfaction: integer (nullable = true)



In [0]:
patients_df.show(5, truncate = False)

+------------+-----------------+---+------------+--------------+----------------+------------+
|patient_id  |name             |age|arrival_date|departure_date|service         |satisfaction|
+------------+-----------------+---+------------+--------------+----------------+------------+
|PAT-09484753|Richard Rodriguez|24 |2025-03-16  |2025-03-22    |surgery         |61          |
|PAT-f0644084|Shannon Walker   |6  |2025-12-13  |2025-12-14    |surgery         |83          |
|PAT-ac6162e4|Julia Torres     |24 |2025-06-29  |2025-07-05    |general_medicine|83          |
|PAT-3dda2bb5|Crystal Johnson  |32 |2025-10-12  |2025-10-23    |emergency       |81          |
|PAT-08591375|Garrett Lin      |25 |2025-02-18  |2025-02-25    |ICU             |76          |
+------------+-----------------+---+------------+--------------+----------------+------------+
only showing top 5 rows


In [0]:
patients_df.createOrReplaceTempView('view_patients')

In [0]:
%sql
SELECT * FROM view_patients LIMIT 5

patient_id,name,age,arrival_date,departure_date,service,satisfaction
PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61
PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83
PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83
PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81
PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76


In [0]:
patients_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_patients')

In [0]:
%sql
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_patients


COUNT(*)
1000


In [0]:
%sql
DESCRIBE TABLE workspace.hospital_beds_db.tbl_patients

col_name,data_type,comment
patient_id,string,null
name,string,null
age,int,null
arrival_date,date,null
departure_date,date,null
service,string,null
satisfaction,int,null


In [0]:
%sql
SELECT satisfaction 
FROM workspace.hospital_beds_db.tbl_patients
WHERE service = 'emergency'
ORDER BY satisfaction DESC
LIMIT 5

satisfaction
99
99
99
99
99


In [0]:
services_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/services_weekly.csv")

In [0]:
services_df.printSchema()

root
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- available_beds: integer (nullable = true)
 |-- patients_request: integer (nullable = true)
 |-- patients_admitted: integer (nullable = true)
 |-- patients_refused: integer (nullable = true)
 |-- patient_satisfaction: integer (nullable = true)
 |-- staff_morale: integer (nullable = true)
 |-- event: string (nullable = true)



In [0]:
services_df.show(5, truncate = False)

+----+-----+----------------+--------------+----------------+-----------------+----------------+--------------------+------------+-----+
|week|month|service         |available_beds|patients_request|patients_admitted|patients_refused|patient_satisfaction|staff_morale|event|
+----+-----+----------------+--------------+----------------+-----------------+----------------+--------------------+------------+-----+
|1   |1    |emergency       |32            |76              |32               |44              |67                  |70          |none |
|1   |1    |surgery         |45            |130             |45               |85              |83                  |78          |flu  |
|1   |1    |general_medicine|37            |201             |37               |164             |97                  |43          |flu  |
|1   |1    |ICU             |22            |31              |22               |9               |84                  |91          |flu  |
|2   |1    |emergency       |28          

In [0]:
services_df.createOrReplaceTempView('view_services')

In [0]:
%sql
SELECT * FROM view_services LIMIT 5

week,month,service,available_beds,patients_request,patients_admitted,patients_refused,patient_satisfaction,staff_morale,event
1,1,emergency,32,76,32,44,67,70,none
1,1,surgery,45,130,45,85,83,78,flu
1,1,general_medicine,37,201,37,164,97,43,flu
1,1,ICU,22,31,22,9,84,91,flu
2,1,emergency,28,169,28,141,75,64,none


In [0]:
services_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_services')

In [0]:
%sql
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_services

COUNT(*)
208


In [0]:
%sql
DESCRIBE TABLE workspace.hospital_beds_db.tbl_services

col_name,data_type,comment
week,int,null
month,int,null
service,string,null
available_beds,int,null
patients_request,int,null
patients_admitted,int,null
patients_refused,int,null
patient_satisfaction,int,null
staff_morale,int,null
event,string,null


In [0]:
%sql
SELECT patients_admitted, patients_refused
FROM workspace.hospital_beds_db.tbl_services
WHERE service = 'emergency'
ORDER BY patients_admitted DESC
LIMIT 5

patients_admitted,patients_refused
36,151
34,164
33,94
32,145
32,44


In [0]:
staff_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/staff.csv")

In [0]:
staff_df.printSchema()

root
 |-- staff_id: string (nullable = true)
 |-- staff_name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- service: string (nullable = true)



In [0]:
staff_df.show(5, truncate = False)

+------------+---------------+------+---------+
|staff_id    |staff_name     |role  |service  |
+------------+---------------+------+---------+
|STF-5ca26577|Allison Hill   |doctor|emergency|
|STF-02ae59ca|Noah Rhodes    |doctor|emergency|
|STF-d8006e7c|Angie Henderson|doctor|emergency|
|STF-212d8b31|Daniel Wagner  |doctor|emergency|
|STF-107a58e4|Cristian Santos|doctor|emergency|
+------------+---------------+------+---------+
only showing top 5 rows


In [0]:
staff_df.createOrReplaceTempView('staff')

In [0]:
%sql
SELECT * FROM staff LIMIT 5

staff_id,staff_name,role,service
STF-5ca26577,Allison Hill,doctor,emergency
STF-02ae59ca,Noah Rhodes,doctor,emergency
STF-d8006e7c,Angie Henderson,doctor,emergency
STF-212d8b31,Daniel Wagner,doctor,emergency
STF-107a58e4,Cristian Santos,doctor,emergency


In [0]:
staff_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_staff')

In [0]:
%sql
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_staff

COUNT(*)
110


In [0]:
%sql
DESCRIBE TABLE workspace.hospital_beds_db.tbl_staff

col_name,data_type,comment
staff_id,string,null
staff_name,string,null
role,string,null
service,string,null


In [0]:
%sql
SELECT DISTINCT role
FROM workspace.hospital_beds_db.tbl_staff

role
doctor
nurse
nursing_assistant


In [0]:
schedule_df = spark.read\
    .option('header', True)\
    .option('delimiter', ',')\
    .option('inferSchema', True)\
    .csv("/Volumes/workspace/hospital_beds_db/hospital_beds_volume/staff_schedule.csv")

In [0]:
schedule_df.printSchema()

root
 |-- week: integer (nullable = true)
 |-- staff_id: string (nullable = true)
 |-- staff_name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- service: string (nullable = true)
 |-- present: integer (nullable = true)



In [0]:
schedule_df.show(5, truncate = False)

+----+------------+------------+------+---------+-------+
|week|staff_id    |staff_name  |role  |service  |present|
+----+------------+------------+------+---------+-------+
|1   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
|2   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
|3   |STF-b77cdc60|Allison Hill|doctor|emergency|0      |
|4   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
|5   |STF-b77cdc60|Allison Hill|doctor|emergency|1      |
+----+------------+------------+------+---------+-------+
only showing top 5 rows


In [0]:
schedule_df.createOrReplaceTempView('schedule')

In [0]:
%sql
SELECT * FROM schedule LIMIT 5

week,staff_id,staff_name,role,service,present
1,STF-b77cdc60,Allison Hill,doctor,emergency,1
2,STF-b77cdc60,Allison Hill,doctor,emergency,1
3,STF-b77cdc60,Allison Hill,doctor,emergency,0
4,STF-b77cdc60,Allison Hill,doctor,emergency,1
5,STF-b77cdc60,Allison Hill,doctor,emergency,1


In [0]:
schedule_df.write.mode('overwrite').saveAsTable('workspace.hospital_beds_db.tbl_schedule')

In [0]:
%sql
SELECT COUNT(*) FROM workspace.hospital_beds_db.tbl_schedule

COUNT(*)
6552


In [0]:
%sql
DESCRIBE TABLE workspace.hospital_beds_db.tbl_schedule

col_name,data_type,comment
week,int,null
staff_id,string,null
staff_name,string,null
role,string,null
service,string,null
present,int,null


In [0]:
%sql
SELECT DISTINCT service
FROM workspace.hospital_beds_db.tbl_schedule

service
emergency
surgery
general_medicine
ICU
